* __Задание 0__: выбрать любую модель машнного обучения и зафиксировать любой тип валидации. Обучить базовую модель и зафиксировать базовое качество модели. В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага.

* __Задание 1__: признак TransactionDT - это смещение в секундах относительно базовой даты. Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

* __Задание 2__: сгруппировать данные по card1 и посчитать среднюю сумму транзакции. Добавить в качестве признака в набор данных. Посчитать разницу между суммой транзакцией пользователя и средней суммой транзакции по данному типу card1. Построить отношение этих признаков. Повторить процедуру для всех card.

* __Задание 3__: преобразовать признаки card_1 - card_6 с помощью Frequency Encoding;

* __Задание 4__: преобразовать признак TransactionAmt в логариф признака, выделить дробную часть и целую часть в отдельные признаки.

* __Задание 5__: для числовых признаков построить PCA-признаки, добавить их к основной части датасета.

* __Задание 6__: использовать критерий отбора признаков на основе перестановок для отбора признаков, которые положительно влияют на перформанс модели. Переобучить модель и сделать выводы о полученном качестве алгоритма.

* __Задание 7__: (опциональное) испольовать SHAP для оценки важности признаков. Использовать SHAP для отбора признаков, проанализировать поведение модели на одном (любом) примере, посмотреть какой вклад каждый признак вносит в итоговый прогно. Проанализировать резульаты.

In [63]:
import os
from copy import deepcopy
from typing import List, Tuple

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import probplot

import catboost as cb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, KFold
import datetime
from typing import List, Optional

In [2]:
train = pd.read_csv("assignment_train.csv")
test = pd.read_csv("assignment_test.csv")
train.head(n=2)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def fit_catboost(x_train, y_train, model_params, categorical, *args):
    """
    Обучение модели CatBoostClassifier.

    Parameters
    ----------
    x_train: pandas.core.frame.DataFrame
        Матрица признаков для обучения модели.

    y_valid: pandas.core.frame.Series
        Вектор целевой переменной для валидации модели.

    model_params: dict
        Словарь со значением гиперпараметров модели.

    categorical: List[str]
        Список с названием категориальных признаков.

    Returns
    -------
    model: catboost.core.CatBoostClassifier
        Экземпляр обученного классификатора.

    """
    eval_set = [(x_train, y_train)]

    if args == 2:
        eval_set.append((args[0], args[1]))

    model = cb.CatBoostClassifier(**model_params)
    model.fit(
        X=x_train,
        y=y_train,
        eval_set=eval_set,
        cat_features=categorical
    )

    return model

def evaluate_model(model, *args):
    """
    Оценка качества модели.

    Parameters
    ----------
    model: catboost.core.CatBoostClassifier
        Экземпляр обученного классификатора.

    args: pandas.core.frame.DataFrame
        Пары из обучающей выборки и вектора истинных ответов.
        Опциональные параметры.

    """
    eval_data = [(args[i], args[i+1]) for i in range(0, len(args), 2)]
    if eval_data:
        for sample, target in eval_data:
            y_pred = model.predict_proba(sample)[:, 1]
            score = roc_auc_score(target, y_pred)
            print(f"score = {round(score, 6)}")


In [4]:
to_drop = [
    "TransactionID",
    "TransactionDT",
    "isFraud",
]

categorical = train.select_dtypes(include=["object"]).columns.tolist()

In [18]:
def extract_date_feauters(data):
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['dayofmonth'] = data['date'].dt.day
    data['dayofweek'] = data['date'].dt.dayofweek
    data['hour'] = data['date'].dt.hour
    return data

In [64]:
def create_categorical_aggs(data: pd.DataFrame,
                            groupby_id: str,
                            features: List[str],
                            prefix: Optional[str] = None,
                            suffix: Optional[str] = None,
                            ) -> pd.DataFrame:
    """
    Построение агрегаций для категориальных признаков.
    Для категориальных признако считаются счетчики для
    каждого значения категории и среднее значение счетчика
    для каждого значения категории.

    Parameters
    ----------
    data: pandas.core.frame.DataFrame
        Выборка для построения агрегаций.

    groupby_id: str
        Название ключа, по которому нужно произвести группировку.

    features: List[str]
        Список с названием признаков, для которых произвести
        группировку.

    prefix: str, optional, default = None
        Префикс для названия признаков.
        Опциональный параметр, по умолчанию, не используется.

    suffix: str, optional, default = None
        Суффикс для названия признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    stats: pandas.core.frame.DataFrame
        Выборка с рассчитанными агрегациями.

    """
    if not prefix:
        prefix = ""
    if not suffix:
        suffix = ""

    categorical = pd.get_dummies(data[features])
    columns_to_agg = categorical.columns

    categorical[groupby_id] = data[groupby_id]
    data_grouped = categorical.groupby(groupby_id)
    stats = data_grouped.agg({col: ["mean", "sum"] for col in columns_to_agg})
    stats.columns = [f"{prefix}{feature}_{stat}{suffix}".upper() for feature, stat in stats]
    stats.columns = [col.replace("MEAN", "RATIO") for col in stats.columns]
    stats.columns = [col.replace("SUM", "TOTAL") for col in stats.columns]
    stats = stats.reset_index()

    return stats

In [50]:
def prepare_data(X, categorical, to_drop, param=0):
    """
    Преобразование данных для передачи в модель.

    Parameters
    ----------
    X: pandas.core.frame.DataFrame
        Матрица признаков для передачи в модель.

    categorical: List[str]
        Список с названием категориальных признаков.

    drop_features: List[str]
        Список с названием признаков, которые не должны
        участвовать в обучении.
    
    param: номер задания

    Returns
    -------
    X_transformed: pandas.core.frame.DataFrame
        Матрица признаков, подготовленная для передачи в модель.

    """
    X_transformed = X.copy()
    
    if param==1:  
        X_transformed['date'] = [datetime.datetime(2017, 12, 1, 0, 0, 0) + datetime.timedelta(seconds=s) for s in X_transformed.TransactionDT]
        X_transformed = extract_date_feauters(X_transformed)
        X_transformed = X_transformed.drop('date', axis=1)
    
    elif param==2:
        aggs = {
            "card1": [np.mean, np.std, np.sum, "max"],
            "card2": [np.mean]
        }

        stats = create_numerical_aggs(apps, groupby_id="APPLICATION_NUMBER", aggs=aggs, prefix="PREV_APP_")
    
    to_drop = set(X.columns) & set(to_drop)

    if to_drop:
        X_transformed = X_transformed.drop(to_drop, axis=1)

    X_transformed[categorical] = X_transformed[categorical].astype(str)
    
    return X_transformed

In [22]:
def ExecuteTask(train, test, number=0):
    
    """
    Выполняем задание.

    Parameters
    ----------
    train: pandas.core.frame.DataFrame
        Трейн.

    test: pandas.core.frame.DataFrame
        Тест.
    
    param: номер задания
    
    """
    
    x_train, x_valid = train_test_split(train, train_size=0.6, random_state=27, shuffle=True)
    y_train, y_valid = train_test_split(train["isFraud"], train_size=0.6, random_state=27, shuffle=True)

    x_valid, x_test = train_test_split(x_valid, train_size=0.5, random_state=72, shuffle=True)
    y_valid, y_test = train_test_split(y_valid, train_size=0.5, random_state=72, shuffle=True)
    
    x_train = prepare_data(x_train, categorical=categorical, to_drop=to_drop, param=number)
    x_valid = prepare_data(x_valid, categorical=categorical, to_drop=to_drop, param=number)
    x_test = prepare_data(x_test, categorical=categorical, to_drop=to_drop, param=number)
    x_public_lb = prepare_data(test, categorical=categorical, to_drop=to_drop, param=number)
    
    cb_params_1000 = {
        "n_estimators": 1000,
        "learning_rate": 0.01,
        "loss_function": "Logloss",
        "eval_metric": "AUC",
        "task_type": "CPU",
        "max_bin": 20,
        "verbose": 10,
        "max_depth": 6,
        "l2_leaf_reg": 100,
        "early_stopping_rounds": 50,
        "thread_count": 6,
        "random_seed": 27
    }
    
    model = fit_catboost(
    x_train, y_train, cb_params_1000, categorical, x_valid, y_valid)
    
    evaluate_model(model, x_train, y_train, x_valid, y_valid, x_test, y_test, x_public_lb, test["isFraud"])
    

In [25]:
ExecuteTask(train, test, 0)

0:	test: 0.6329804	best: 0.6329804 (0)	total: 183ms	remaining: 3m 2s
10:	test: 0.7618445	best: 0.7653676 (8)	total: 1.74s	remaining: 2m 36s
20:	test: 0.7930679	best: 0.7930679 (20)	total: 3.28s	remaining: 2m 32s
30:	test: 0.7928800	best: 0.7962874 (24)	total: 4.74s	remaining: 2m 28s
40:	test: 0.8037710	best: 0.8040604 (32)	total: 6.32s	remaining: 2m 27s
50:	test: 0.8181301	best: 0.8181301 (50)	total: 7.9s	remaining: 2m 27s
60:	test: 0.8192070	best: 0.8192070 (60)	total: 9.49s	remaining: 2m 26s
70:	test: 0.8255115	best: 0.8259605 (69)	total: 11.1s	remaining: 2m 24s
80:	test: 0.8265871	best: 0.8265871 (80)	total: 12.6s	remaining: 2m 23s
90:	test: 0.8286209	best: 0.8286209 (90)	total: 14.2s	remaining: 2m 21s
100:	test: 0.8336715	best: 0.8336715 (100)	total: 15.8s	remaining: 2m 20s
110:	test: 0.8369921	best: 0.8369921 (110)	total: 17.4s	remaining: 2m 18s
120:	test: 0.8414312	best: 0.8414312 (120)	total: 18.9s	remaining: 2m 17s
130:	test: 0.8452370	best: 0.8452370 (130)	total: 20.5s	remaini

In [51]:
ExecuteTask(train, test, 1)

0:	test: 0.6485707	best: 0.6485707 (0)	total: 182ms	remaining: 3m 2s
10:	test: 0.7827775	best: 0.7827775 (10)	total: 1.85s	remaining: 2m 46s
20:	test: 0.8030880	best: 0.8032186 (18)	total: 3.32s	remaining: 2m 34s
30:	test: 0.8134770	best: 0.8134770 (30)	total: 4.79s	remaining: 2m 29s
40:	test: 0.8163637	best: 0.8166673 (38)	total: 6.35s	remaining: 2m 28s
50:	test: 0.8204018	best: 0.8205203 (49)	total: 7.86s	remaining: 2m 26s
60:	test: 0.8203998	best: 0.8207873 (52)	total: 9.36s	remaining: 2m 24s
70:	test: 0.8228372	best: 0.8229957 (66)	total: 10.9s	remaining: 2m 22s
80:	test: 0.8312177	best: 0.8312177 (80)	total: 12.4s	remaining: 2m 20s
90:	test: 0.8349627	best: 0.8349627 (90)	total: 13.9s	remaining: 2m 18s
100:	test: 0.8403928	best: 0.8403928 (100)	total: 15.4s	remaining: 2m 17s
110:	test: 0.8460457	best: 0.8460457 (110)	total: 17s	remaining: 2m 15s
120:	test: 0.8471919	best: 0.8477113 (115)	total: 18.5s	remaining: 2m 14s
130:	test: 0.8511192	best: 0.8511192 (130)	total: 20s	remaining

Вывод: качество изменилось незначительно. Можно было и не делать такое преобразование.

In [54]:
train.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
count,5.000100e+04,50001.000000,5.000100e+04,50001.000000,50001.000000,49304.000000,49998.000000,49766.000000,47410.000000,47410.000000,...,12268.000000,12268.000000,12268.000000,12268.000000,12268.000000,12268.000000,12268.000000,12268.000000,12268.000000,12268.000000
mean,3.012000e+06,0.027139,6.378101e+05,128.129726,9912.598368,366.635547,152.803612,200.704276,291.813162,85.590593,...,0.110450,38.942391,63.051814,45.816241,4.252527,14.926426,7.991718,15.742603,21.362671,16.464481
std,1.443419e+04,0.162491,3.229461e+05,213.078011,4904.597980,159.057800,10.482210,40.299048,104.991697,6.687334,...,0.469788,190.465893,446.405097,210.319262,46.694625,105.561868,60.584452,91.083819,131.071046,92.726771
min,2.987000e+06,0.000000,8.640000e+04,0.292000,1001.000000,100.000000,100.000000,100.000000,100.000000,13.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.999500e+06,0.000000,3.523050e+05,42.000000,6019.000000,208.000000,150.000000,166.000000,204.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.012000e+06,0.000000,6.537500e+05,68.500000,9705.000000,369.000000,150.000000,226.000000,299.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.024500e+06,0.000000,9.321340e+05,120.000000,14265.000000,514.000000,150.000000,226.000000,330.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.037000e+06,1.000000,1.189349e+06,4829.950000,18393.000000,600.000000,229.000000,237.000000,540.000000,102.000000,...,7.000000,6300.000000,32250.000000,6300.000000,1000.000000,2700.000000,1300.000000,1700.000000,3000.000000,1700.000000


In [56]:
train.columns.to_list()

['TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'dist2',
 'P_emaildomain',
 'R_emaildomain',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'V31',
 'V32',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V39',
 'V40',
 'V41',
 'V42',
 'V43',
 'V44',
 'V45',
 'V46',
 'V47',
 'V48',
 'V49',
 'V50',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V

In [60]:
train['card4'].value_counts()

visa                32562
mastercard          15696
american express     1083
discover              654
Name: card4, dtype: int64

In [61]:
train['card6'].value_counts()

debit              34206
credit             15786
debit or credit        3
charge card            3
Name: card6, dtype: int64

In [66]:
stats = create_categorical_aggs(
    train, groupby_id="card4", features=["TransactionAmt"], prefix=""
)
stats.head(n=2)

,card4,TRANSACTIONAMT_RATIO,TRANSACTIONAMT_TOTAL
0,american express,167.022392,180885.25
1,discover,211.559924,138360.19
